<a href="https://colab.research.google.com/github/anastasiiansm/AImusic/blob/main/remove_duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: mount the drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

%pylab inline
import pandas as pd
import glob
import re
from scipy import sparse
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

from tqdm.notebook import tqdm
import nltk

tqdm.pandas()


Populating the interactive namespace from numpy and matplotlib


In [5]:
data_file_path = 'data.csv'
data = pd.read_csv(data_file_path)

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your dataset
data_file_path = '/content/data.csv'  # Path to your CSV file
song_lyrics = pd.read_csv(data_file_path)

# Ensure the required columns exist
if 'Title & Artist' not in song_lyrics.columns:
    raise ValueError("The column 'Title & Artist' is required.")

# Count the number of songs per Title & Artist
song_counts = song_lyrics['Title & Artist'].value_counts()

# Convert to DataFrame for better visualization
song_counts_df = song_counts.reset_index()
song_counts_df.columns = ['Title & Artist', 'Number of Songs']

# Display the total number of songs and the first few rows
print('Total number of song lyrics: ', song_lyrics.shape[0])

Total number of song lyrics:  500


In [10]:
# Check if Title & Artist combination is unique
is_duplicate_title_artist = song_lyrics['Title & Artist'].duplicated().any()

if is_duplicate_title_artist:
    print("There are duplicate entries for Title & Artist in the dataset.")
else:
    print("All Title & Artist entries are unique.")

There are duplicate entries for Title & Artist in the dataset.


In [11]:
# Count how many exact duplicate Title & Artist combinations are in the dataset
duplicate_title_artist_count = song_lyrics['Title & Artist'].duplicated().sum()

print(f"There are {duplicate_title_artist_count} exact duplicated Title & Artist combinations in the dataset.")


There are 45 exact duplicated Title & Artist combinations in the dataset.


In [12]:
# Identify exact duplicated Title & Artist combinations
exact_duplicated_title_artist = song_lyrics[song_lyrics['Title & Artist'].duplicated(keep=False)]

# Remove duplicates based on the Title & Artist combination, keeping the first occurrence
song_lyrics = song_lyrics.sort_values("Year").drop_duplicates(subset=['Title & Artist'], keep='first')

# Print the results
print('N songs: ', song_lyrics.shape[0])
print("N unique Title & Artist with exact duplicates: ", exact_duplicated_title_artist.drop_duplicates(subset=['Title & Artist']).shape[0])


N songs:  455
N unique Title & Artist with exact duplicates:  42


In [13]:
# Reset index and set its name to "row"
song_lyrics = song_lyrics.reset_index(drop=True)  # Reset index without adding the old index as a column
song_lyrics.index.name = 'row'  # Name the index as "row"

# Display the first 2 rows of the updated DataFrame
song_lyrics.head(2)


,Language,# in a chart,Title Lyrics,artist_type,Artist Name,Title & Artist,Text Lyrics,Appropriate lyric,Year,Tags,...,audioset_vggish_mood_aggressive_audioset_vggish V2,audioset_vggish_mood_happy_audioset_vggish V2,audioset_vggish_mood_party_audioset_vggish V2,audioset_vggish_mood_relaxed_audioset_vggish V2,audioset_vggish_mood_sad_audioset_vggish V2,audioset_vggish_mood_acoustic_audioset_vggish V2,audioset_vggish_mood_electronic_audioset_vggish V2,audioset_vggish_voice_instrumental_audioset_vggish V2,audioset_vggish_gender_audioset_vggish V2,audioset_vggish_tonality_audioset_vggish V2
row,,,,,,,,,,,,,,,,,,,,,
0,English,100,Robbery,person,Juice WRLD,Robbery by Juice WRLD,"[Intro]\nMan, what?\nThis shit funny, one seco...",NaN,2019,Rap\nUSA\nIn English\nHip-Hop\nEast Coast Rap\...,...,0.927666,0.413556,0.393889,0.505226,0.541307,0.696688,0.765799,0.979119,0.762104,0.151082
1,English,27,Someone You Loved,person,Lewis Capaldi,Someone You Loved by Lewis Capaldi,"[Verse 1]\nI'm going under, and this time, I f...",NaN,2019,Pop\nIn English\nScotland\nAlternative Pop\nSo...,...,0.970194,0.626135,0.211330,0.763995,0.827951,0.404034,0.718109,0.932337,0.340769,0.332026
